In [1]:
import pandas as pd
import numpy as np

import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

In [2]:
df = pd.read_csv('/Users/alejandropalacios/Desktop/Ironhack/Data Analytics Bootcamp/Advanced Data Analysis Techniques/Project/F1-Grand-Prix-Predictor/Data Manipulation/Modeling/Modeled Databases/driver_change_dataset.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13720 entries, 0 to 13719
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   constructor                        13720 non-null  object 
 1   season                             13720 non-null  int64  
 2   round                              13720 non-null  int64  
 3   circuit_id                         13720 non-null  object 
 4   weather_warm                       13720 non-null  bool   
 5   weather_cold                       13720 non-null  bool   
 6   weather_dry                        13720 non-null  bool   
 7   weather_wet                        13720 non-null  bool   
 8   weather_cloudy                     13720 non-null  bool   
 9   driver                             13720 non-null  object 
 10  nationality                        13720 non-null  object 
 11  grid                               13720 non-null  int

In [3]:
# There are two things that must be taken into account before running a WOE and IV process. 

# 1. Data must be CLEAN and FULL, no NULL VALUES: 

df = df.fillna(0)

# 2. There should not be any continuous attributes. Attributes which represent age or anything continuous should be binned into 5–10 bins:

max_bin = 5
force_bin = 5

In [4]:
# Now we will set our target variable.

df['target'] = df['change_drivers']
df = df.drop('change_drivers',axis=1)


In [5]:
# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [6]:
final_iv, IV = data_vars(df,df.target)

In [7]:
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,constructor,ags,ags,41,5,0.121951,36,0.878049,0.009058,0.002734,1.197916,0.337368
1,constructor,alfa,alfa,129,6,0.046512,123,0.953488,0.010870,0.009341,0.151572,0.337368
2,constructor,arrows,arrows,407,20,0.049140,387,0.950860,0.036232,0.029389,0.209304,0.337368
3,constructor,ats,ats,30,2,0.066667,28,0.933333,0.003623,0.002126,0.532940,0.337368
4,constructor,bar,bar,198,6,0.030303,192,0.969697,0.010870,0.014581,-0.293739,0.337368
...,...,...,...,...,...,...,...,...,...,...,...,...
415,driver_age,17,25,3829,212,0.055367,3617,0.944633,0.384058,0.274681,0.335183,0.063040
416,driver_age,26,28,3193,125,0.039148,3068,0.960852,0.226449,0.232989,-0.028471,0.063040
417,driver_age,29,32,3704,126,0.034017,3578,0.965983,0.228261,0.271719,-0.174280,0.063040
418,driver_age,33,43,2994,89,0.029726,2905,0.970274,0.161232,0.220611,-0.313555,0.063040


In [8]:
IV.sort_values('IV')

,VAR_NAME,IV
24,wins_percentage,0.000000
22,weather_warm,0.000001
20,weather_cold,0.004135
11,driver_wins,0.006613
12,driver_wins_after_race.1,0.008239
23,weather_wet,0.008712
19,weather_cloudy,0.016528
21,weather_dry,0.043525
6,driver_age,0.063040
15,podium,0.066968


In [9]:
""" 

Important notes:

By the Sturges Rule, the number of groups or classes is 1 + 3.3 log n, where n is the number of observations. Thus, using the number of our registrations:

1 + 3.3 log(13720) = 5.1373541114

This means we should be using 5 bins for our WOE and IV Studies. 

Moreover, the following Variables will be used based on our results (in order of Predictive Power):

- nationality
- grid	
- qualifying_time
- constructor
- constructor_wins
- constructor_standings_pos
- driver

"""

' \n\nImportant notes:\n\nBy the Sturges Rule, the number of groups or classes is 1 + 3.3 log n, where n is the number of observations. Thus, using the number of our registrations:\n\n1 + 3.3 log(13720) = 5.1373541114\n\nThis means we should be using 5 bins for our WOE and IV Studies. \n\nMoreover, the following Variables will be used based on our results (in order of Predictive Power):\n\n- constructor_standings_pos\n- constructor_wins\n- driver\n- qualifying_time\n- constructor\n- podium\n\n'